In [1]:
## production for retrieving singficant snos and then calculating 
# 1. allele frequenciy from them 
# 2. ecotype frequences 

In [38]:
import os
import numpy as np
import cvxpy as cp
import pickle
import pandas as pd
import dask.dataframe as dd


In [2]:
# Function to load data from a pickle file
def load_data(file_path, variable):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return set(data[variable])

In [3]:
split_number = 1

In [15]:
snps_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/significant_snps.pkl'
beta_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/effect_sizes_simple_full_genome.csv'
new_env_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/testing_environment_lea_full_genome.csv'
genotype_counts_file = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/genotype_counts_fullgenome231ecotypes.csv'

In [5]:
betas = pd.read_csv(beta_file)
new_env = pd.read_csv(new_env_file)

In [6]:
betas

,Unnamed: 0,V1,V2
0,1,0.002006,0.010706
1,2,-0.012903,-0.049465
2,3,0.008464,0.004360
3,4,-0.004655,0.002459
4,5,0.001385,-0.003268
...,...,...,...
3235475,3235476,-0.002292,-0.002130
3235476,3235477,-0.000817,-0.000521
3235477,3235478,0.000321,0.000065
3235478,3235479,0.039891,0.031311


In [7]:
## import the selected snps
snps_v1 = load_data(snps_file, 'V1')
snps_v2 = load_data(snps_file, 'V2')

In [8]:
## filter the betas based on significant alleles 
betas_v1 = betas[betas['Unnamed: 0'].isin(snps_v1)]['V1']
betas_v2 = betas[betas['Unnamed: 0'].isin(snps_v2)]['V2']

In [9]:
## get the 0 indexed snps indexed for future use
snps_v1_0index = pd.Series(list(snps_v1)) - 1 
snps_v2_0index = pd.Series(list(snps_v2)) - 1 

In [32]:
betas_v1

1268      -0.051554
1617      -0.051554
1819      -0.052861
4163      -0.066539
4168      -0.060429
             ...   
3234055    0.020034
3234145    0.020248
3234167    0.027116
3234179    0.020248
3234403    0.020248
Name: V1, Length: 2896, dtype: float64

In [10]:
## calculate the predicted allele frequencies in the new environment by multypling the 2 vectors
pred_af_v1 = np.outer(new_env, betas_v1.T)

In [11]:
pred_af_v1

array([[-0.        , -0.        , -0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.10310718, -0.10310718, -0.10572217, ...,  0.05423179,
         0.04049654,  0.04049654],
       [-0.02079461, -0.02079461, -0.021322  , ...,  0.01093744,
         0.00816732,  0.00816732],
       ...,
       [-2.47457239, -2.47457239, -2.5373321 , ...,  1.30156306,
         0.97191706,  0.97191706],
       [ 0.06775029,  0.06775029,  0.06946857, ..., -0.03563496,
        -0.02660971, -0.02660971],
       [-0.04021554, -0.04021554, -0.04123548, ...,  0.02115236,
         0.01579512,  0.01579512]])

In [16]:
genotype_counts = dd.read_csv(genotype_counts_file, sep = ',')

In [18]:
genotype_counts = genotype_counts.compute()

In [22]:
genotype_counts = genotype_counts.drop('Unnamed: 0',axis=1).reset_index(drop=True)

In [30]:
genotype_countsf = genotype_counts.iloc[list(snps_v1_0index), :]

In [34]:
genotype_countsf = genotype_countsf.sort_index()

In [37]:
## make sure the index (snps indexs) are equal in the genotype_countsf and in the betas_v1
(betas_v1.index == genotype_countsf.index).all()

True

In [48]:
pred_af_v1[0,:].shape

(2896,)

In [49]:
first_sample = pred_af_v1[0,:]

In [54]:
genotype_countsf = np.array(genotype_countsf)

In [66]:
genotype_countsf.shape

(2896, 231)

In [73]:
(genotype_countsf == 3).sum()

0

In [68]:
pred_af_v1.shape

(232, 2896)

In [62]:
## so for pred_af_v1 there are 232 samples we need to predict ecotpye freq at each 
ecotypes_frequency = {}
for i in range(pred_af_v1.shape[0]):

    one_sample = pred_af_v1[i,:]
    y = np.asarray(one_sample) * 2
    
    h = cp.Variable(231)
    
    product = genotype_countsf @ h 
    
    diff = product - y
    
    constraints = [0 <= h, sum(h) == 1]
    
    problem = cp.Problem(cp.Minimize(cp.norm(diff)), constraints)
    
    # Solve the optimization problem using the SCS solver, with verbose output disabled.
    problem.solve(solver='SCS', verbose=False)
    print(i)
    ecotypes_frequency[i] = h.value

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
Failure:interrupted


SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

In [74]:
h.value

In [76]:
ecotypes_frequencydf = pd.DataFrame(ecotypes_frequency)

In [83]:
ecotypes_frequencydf[0].sort_values()

62    -0.000178
109   -0.000064
22    -0.000062
20    -0.000060
117   -0.000059
         ...   
35     0.040730
203    0.041568
71     0.058981
75     0.074366
21     0.550191
Name: 0, Length: 231, dtype: float64

In [80]:
ecotypes_frequencydf.sum(axis=0).head(50)

0     1.000000
1     1.000000
2     1.000000
3     0.999999
4     1.000002
5     1.000000
6     1.000000
7     0.999999
8     1.000000
9     1.000000
10    1.000000
11    0.999999
12    1.000000
13    1.000000
14    1.000000
15    0.999999
16    0.999998
17    1.000000
18    1.000000
19    0.999999
20    1.000000
21    1.000000
22    1.000000
23    0.999999
24    1.000000
25    1.000000
26    1.000000
27    0.999999
28    0.999998
29    1.000000
30    0.999999
31    1.000002
32    1.000000
33    1.000000
34    0.999999
35    1.000002
36    0.999999
37    1.000000
38    0.999999
39    1.000002
40    1.000000
41    1.000000
42    0.999999
43    1.000002
44    1.000000
45    1.000000
46    0.999999
47    1.000002
48    0.999999
49    1.000000
dtype: float64